In [1]:
from __future__ import print_function
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.8000, 0.5100, 0.3122],
        [0.3300, 0.0779, 0.5332],
        [0.3980, 0.7915, 0.6994],
        [0.7075, 0.4491, 0.0516],
        [0.0350, 0.1723, 0.9024]])


In [2]:
import torch
torch.cuda.is_available()

True

In [4]:
import torch

print(torch.cuda.current_device())

print(torch.cuda.device(0))

print(torch.cuda.device_count())

print(torch.cuda.get_device_name(0))

0
1
Quadro M2200


In [15]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time

t0 = time.time()

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 20
learning_rate = 0.01

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

t1 = time.time()

print(t1-t0)
# 220.4247305393219

Epoch [1/20], Step [100/600], Loss: 0.6085
Epoch [1/20], Step [200/600], Loss: 0.1974
Epoch [1/20], Step [300/600], Loss: 0.1337
Epoch [1/20], Step [400/600], Loss: 0.1781
Epoch [1/20], Step [500/600], Loss: 0.1432
Epoch [1/20], Step [600/600], Loss: 0.2027
Epoch [2/20], Step [100/600], Loss: 0.1270
Epoch [2/20], Step [200/600], Loss: 0.0575
Epoch [2/20], Step [300/600], Loss: 0.0849
Epoch [2/20], Step [400/600], Loss: 0.0295
Epoch [2/20], Step [500/600], Loss: 0.0455
Epoch [2/20], Step [600/600], Loss: 0.1252
Epoch [3/20], Step [100/600], Loss: 0.0543
Epoch [3/20], Step [200/600], Loss: 0.0418
Epoch [3/20], Step [300/600], Loss: 0.0147
Epoch [3/20], Step [400/600], Loss: 0.1175
Epoch [3/20], Step [500/600], Loss: 0.0273
Epoch [3/20], Step [600/600], Loss: 0.0246
Epoch [4/20], Step [100/600], Loss: 0.0937
Epoch [4/20], Step [200/600], Loss: 0.1405
Epoch [4/20], Step [300/600], Loss: 0.0601
Epoch [4/20], Step [400/600], Loss: 0.0723
Epoch [4/20], Step [500/600], Loss: 0.0991
Epoch [4/20